## Univariate correlation analysis

TODO: describe

In [1]:
import sys
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import pancancer_evaluation.config as cfg
import pancancer_evaluation.utilities.data_utilities as du

%load_ext autoreload
%autoreload 2

In [2]:
gene = 'TP53'
mad_threshold = 100

### Load expression data and mutation label data

In [3]:
print('Loading gene label data...', file=sys.stderr)
genes_df = du.load_top_50()
sample_info_df = du.load_sample_info(verbose=True)

# this returns a tuple of dataframes, unpack it below
pancancer_data = du.load_pancancer_data(verbose=True)
(sample_freeze_df,
 mutation_df,
 copy_loss_df,
 copy_gain_df,
 mut_burden_df) = pancancer_data

rnaseq_df = du.load_expression_data(verbose=True)

# standardize columns of expression dataframe
print('Standardizing columns of expression data...', file=sys.stderr)
rnaseq_df[rnaseq_df.columns] = StandardScaler().fit_transform(rnaseq_df[rnaseq_df.columns])

Loading gene label data...
Loading sample info...
Loading pan-cancer data from cached pickle file...
Loading gene expression data...
Standardizing columns of expression data...


In [4]:
print(rnaseq_df.shape)
rnaseq_df.iloc[:5, :5]

(11060, 16148)


,1,10,100,1000,10000
sample_id,,,,,
TCGA-02-0047-01,-0.144100,-0.136450,-0.207065,1.049402,0.644625
TCGA-02-0055-01,-0.124925,-0.197893,-0.132694,0.704438,0.154763
TCGA-02-2483-01,-0.133543,-0.174587,-0.103291,1.473420,0.669303
TCGA-02-2485-01,-0.147052,-0.072888,-0.213119,4.405612,11.503035
TCGA-02-2486-01,-0.145321,-0.181076,-0.147395,1.013468,0.117745


In [5]:
sample_freeze_df.head()

,PATIENT_BARCODE,SAMPLE_BARCODE,DISEASE,SUBTYPE
0,TCGA-OR-A5J1,TCGA-OR-A5J1-01,ACC,Not_Applicable
1,TCGA-OR-A5J2,TCGA-OR-A5J2-01,ACC,Not_Applicable
2,TCGA-OR-A5J3,TCGA-OR-A5J3-01,ACC,Not_Applicable
3,TCGA-OR-A5J5,TCGA-OR-A5J5-01,ACC,Not_Applicable
4,TCGA-OR-A5J6,TCGA-OR-A5J6-01,ACC,Not_Applicable


In [6]:
mutation_df.iloc[:5, :5]

,5S_rRNA,A1BG,A1CF,A2M,A2ML1
SAMPLE_BARCODE,,,,,
TCGA-02-0047-01,0,0,0,0,0
TCGA-02-0055-01,0,0,0,0,0
TCGA-02-2483-01,0,0,0,0,0
TCGA-02-2485-01,0,0,0,0,0
TCGA-02-2486-01,0,0,0,0,0


In [7]:
y_df = (mutation_df
    .loc[:, [gene]]
    .merge(sample_freeze_df, left_index=True, right_on='SAMPLE_BARCODE')
    .drop(columns='PATIENT_BARCODE')
    .set_index('SAMPLE_BARCODE')
    .rename(columns={gene: 'status',
                     'DISEASE': 'cancer_type',
                     'SUBTYPE': 'subtype'})
)
print(y_df.shape)
y_df.head()

(9074, 3)


,status,cancer_type,subtype
SAMPLE_BARCODE,,,
TCGA-02-0047-01,0,GBM,IDHwt
TCGA-02-0055-01,1,GBM,IDHwt
TCGA-02-2483-01,1,GBM,IDHmut-non-codel
TCGA-02-2485-01,1,GBM,IDHwt
TCGA-02-2486-01,0,GBM,IDHwt


In [8]:
X_df = rnaseq_df.reindex(y_df.index)
print(X_df.shape)
print(X_df.isna().sum().sum())
X_df.iloc[:5, :5]

(9074, 16148)
0


,1,10,100,1000,10000
SAMPLE_BARCODE,,,,,
TCGA-02-0047-01,-0.144100,-0.136450,-0.207065,1.049402,0.644625
TCGA-02-0055-01,-0.124925,-0.197893,-0.132694,0.704438,0.154763
TCGA-02-2483-01,-0.133543,-0.174587,-0.103291,1.473420,0.669303
TCGA-02-2485-01,-0.147052,-0.072888,-0.213119,4.405612,11.503035
TCGA-02-2486-01,-0.145321,-0.181076,-0.147395,1.013468,0.117745


### Subset genes by mean absolute deviation

In [9]:
# first subset by MAD
mad_genes_df = (rnaseq_df
    .mad(axis=0)
    .sort_values(ascending=False)
    .reset_index()
)
mad_genes_df.head()

,index,0
0,126695,0.821090
1,147798,0.808853
2,10053,0.808764
3,54845,0.803626
4,582,0.802497


In [10]:
mad_genes_df.columns=['gene_id', 'mad']
mad_genes = mad_genes_df.iloc[:mad_threshold, :].gene_id.astype(str).values
print(mad_genes[:5])

['126695' '147798' '10053' '54845' '582']


In [11]:
X_df = X_df.reindex(mad_genes, axis='columns')
print(X_df.shape)
X_df.iloc[:5, :5]

(9074, 100)


,126695,147798,10053,54845,582
SAMPLE_BARCODE,,,,,
TCGA-02-0047-01,-1.307892,-0.977865,-1.119059,-1.116767,0.458711
TCGA-02-0055-01,-1.305820,-0.964695,-1.111900,-1.114346,-0.551517
TCGA-02-2483-01,-1.307892,-0.984642,-1.119059,-1.116851,-0.470321
TCGA-02-2485-01,-1.306134,-0.984678,-1.118344,-1.113535,1.100724
TCGA-02-2486-01,-1.305639,-0.983574,-1.119059,-1.117114,1.006310


### Calculate pan-cancer univariate feature correlations

In [12]:
# now get univariate feature correlations with labels
from sklearn.feature_selection import f_classif

f_stats_pancan = f_classif(X_df, y_df.status)[0]
print(f_stats_pancan)

[2.76606990e+02 7.02872243e+01 3.47539437e+01 7.53519282e+02
 7.44360880e+02 4.46192872e+02 1.17054890e+03 1.68578528e+02
 5.27937846e+00 1.50456916e+02 1.69441254e+01 2.83799667e+01
 1.81283292e+02 1.81860047e+02 1.21528596e+03 9.66673163e+01
 9.73105305e+00 3.64416541e+02 2.31692409e+01 4.81917143e+02
 6.16061787e+02 8.16934252e+01 1.57162530e+02 1.06114720e+03
 9.55631077e+01 5.46751089e+02 7.44611032e+02 5.66550812e+02
 5.52401563e+02 2.87701791e+00 2.82099162e+02 1.92113447e+02
 1.49318434e+02 2.46996711e+01 1.57171729e+01 5.72854481e+00
 9.59491476e+01 1.17693128e+01 1.43706915e+01 1.16411845e+02
 2.24732308e+02 3.02421764e+02 4.40544808e+02 6.95960404e+00
 4.69583295e+02 3.21183714e+01 6.30831691e+01 1.32760829e+02
 7.49973212e+02 2.34246195e+00 4.05078465e+02 9.23233478e+02
 1.90317914e+01 8.62842986e+02 1.29750470e+02 1.12404161e+03
 3.81042096e+02 1.56625295e+02 1.35075994e+01 8.21075427e+02
 6.59651258e+02 5.93133112e+02 7.64606987e+02 1.48822318e+02
 2.11273531e+01 2.073767

In [13]:
def get_f_stats_for_cancer_types(X_df, y_df):
    f_stats_df = {}
    for cancer_type in y_df.cancer_type.unique():
        ct_samples = y_df[y_df.cancer_type == cancer_type].index
        X_ct_df = X_df.reindex(ct_samples)
        y_ct_df = y_df.reindex(ct_samples)
        
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            try:
                f_stats_df[cancer_type] = f_classif(X_ct_df, y_ct_df.status)[0]
            except RuntimeWarning:
                # this can happen if there are no mutated samples in the cancer type
                # in that case, just skip it
                continue
        
    return pd.DataFrame(f_stats_df, index=X_df.columns).T

f_stats_df = get_f_stats_for_cancer_types(X_df, y_df)
f_stats_df.loc['pancan'] = f_stats_pancan
print(f_stats_df.shape)
print(f_stats_df.isna().sum().sum())
f_stats_df.iloc[-5:, :5]

(33, 100)
0


,126695,147798,10053,54845,582
KICH,0.481428,0.417635,0.012031,1.429834,0.103799
UCS,0.416370,0.000413,0.005767,0.065855,3.112278
ACC,0.444091,0.914762,0.002503,0.000216,0.218138
PCPG,0.025548,0.026923,0.006727,0.008308,0.014604
pancan,276.606990,70.287224,34.753944,753.519282,744.360880
